
An attempt to measure suprise in text using adapters

https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py

In [1]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
import pandas as pd

/media/wassname/SGIronWolf/projects5/bs_writing_detector/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# from peft import LoraConfig, get_peft_model


# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
MAX_LEN = 2000
import json
samples = json.load(open("../samples.json"))
df_samples = pd.DataFrame(samples)
df_samples['len'] = df_samples['text'].str.len()
df_samples


sample = samples[0]
sample

{'name': 'bad_ml',
 'url': 'https://arxiv.org/abs/2312.10868',
 'text': 'This roadmap survey has embarked on an exploration of the\ntransformative trends in generative AI research, particularly focusing on speculated advancements like Q* and the progressive strides towards AGI. Our analysis highlights a crucial paradigm shift, driven by innovations such as MoE, multi-modal learning, and the pursuit of AGI. These advancements signal a future where AI systems could significantly extend their capabilities in reasoning, contextual understanding, and creative problem-solving. This study reflects on AI’s dual potential to either contribute to or impede global equity and justice. The equitable distribution of AI benefits and its role in decision-making processes raise crucial questions about fairness and inclusivity. It is imperative to thoughtfully integrate AI into societal structures to enhance justice and reduce disparities. Despite these advancements, several open questions and research 

## Helpers

In [4]:
from dotenv import load_dotenv

load_dotenv() 

True

In [5]:
import os
from openai import OpenAI

def summize(text):
    client = OpenAI()
    content = f"Make a tl;dr of this text in <280 chars.\n\n## Text\n\n{text}\n\n## Instruction\n\nMake a tl;dr of this text in <280 chars. Start with the most important, as extra text will be discarded :\n\ntl;dr:"
    chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
            model="gpt-4",
        )
    # print(content)
    r = chat_completion.choices[0].message.content
    return r

r = summize(samples[1]["text"])
r

'Exploring representation engineering (RepE) for AI transparency, we proposed new methods that improved results on TruthfulQA and presented a tool for AI safety issues. Future RepE research could further boost our control over AI systems.'

In [6]:
# modified from https://github.dev/huggingface/evaluate/blob/8dfe05784099fb9af55b8e77793205a3b7c86465/measurements/perplexity/perplexity.py#L154

# from evaluate.measurements.perplexity import Perplexity
import evaluate
from evaluate import logging
from torch.nn import CrossEntropyLoss

# @evaluate.utils.file_utils.add_start_docstrings(_DESCRIPTION, _KWARGS_DESCRIPTION)
def perplexity_compute(
    data, model, tokenizer, batch_size: int = 16, add_start_token: bool = True, device=None, max_length=None
):

    if device is not None:
        assert device in ["gpu", "cpu", "cuda"], "device should be either gpu or cpu."
        if device == "gpu":
            device = "cuda"
    else:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # model = AutoModelForCausalLM.from_pretrained(model_id)
    model = model.to(device)

    # tokenizer = AutoTokenizer.from_pretrained(model_id)

    # if batch_size > 1 (which generally leads to padding being required), and
    # if there is not an already assigned pad_token, assign an existing
    # special token to also be the padding token
    if tokenizer.pad_token is None and batch_size > 1:
        existing_special_tokens = list(tokenizer.special_tokens_map_extended.values())
        # check that the model already has at least one special token defined
        assert (
            len(existing_special_tokens) > 0
        ), "If batch_size > 1, model must have at least one special token to use for padding. Please use a different model or set batch_size=1."
        # assign one of the special tokens to also be the pad token
        tokenizer.add_special_tokens({"pad_token": existing_special_tokens[0]})

    if add_start_token and max_length:
        # leave room for <BOS> token to be added:
        assert (
            tokenizer.bos_token is not None
        ), "Input model must already have a BOS token if using add_start_token=True. Please use a different model, or set add_start_token=False"
        max_tokenized_len = max_length - 1
    else:
        max_tokenized_len = max_length

    encodings = tokenizer(
        data,
        add_special_tokens=False,
        padding=True,
        truncation=True if max_tokenized_len else False,
        max_length=max_tokenized_len,
        return_tensors="pt",
        return_attention_mask=True,
    ).to(device)

    encoded_texts = encodings["input_ids"]
    attn_masks = encodings["attention_mask"]

    # check that each input is long enough:
    if add_start_token:
        assert torch.all(torch.ge(attn_masks.sum(1), 1)), "Each input text must be at least one token long."
    else:
        assert torch.all(
            torch.ge(attn_masks.sum(1), 2)
        ), "When add_start_token=False, each input text must be at least two tokens long. Run with add_start_token=True if inputting strings of only one token, and remove all empty input strings."

    ppls = []
    loss_fct = CrossEntropyLoss(reduction="none")

    for start_index in logging.tqdm(range(0, len(encoded_texts), batch_size)):
        end_index = min(start_index + batch_size, len(encoded_texts))
        encoded_batch = encoded_texts[start_index:end_index]
        attn_mask = attn_masks[start_index:end_index]

        if add_start_token:
            bos_tokens_tensor = torch.tensor([[tokenizer.bos_token_id]] * encoded_batch.size(dim=0)).to(device)
            encoded_batch = torch.cat([bos_tokens_tensor, encoded_batch], dim=1)
            attn_mask = torch.cat(
                [torch.ones(bos_tokens_tensor.size(), dtype=torch.int64).to(device), attn_mask], dim=1
            )

        labels = encoded_batch

        with torch.no_grad():
            out_logits = model(encoded_batch, attention_mask=attn_mask).logits

        shift_logits = out_logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        shift_attention_mask_batch = attn_mask[..., 1:].contiguous()

        perplexity_batch = torch.exp(
            (loss_fct(shift_logits.transpose(1, 2), shift_labels) * shift_attention_mask_batch).sum(1)
            / shift_attention_mask_batch.sum(1)
        )

        ppls += perplexity_batch.tolist()

    return {"perplexities": ppls, "mean_perplexity": np.mean(ppls)}

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoModelForCausalLM, AutoConfig, PreTrainedTokenizerBase, PreTrainedTokenizer, GPTQConfig, BitsAndBytesConfig

def load_model(model_name):
    trust_remote_code = True
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    # print(config)
    if config.quantization_config is not None:
        config.quantization_config['disable_exllama'] = True
        if 'use_exllama' in config.quantization_config:
            del config.quantization_config['use_exllama']
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=trust_remote_code, 
                                                 config=config,
                                                 )
    return model, tokenizer


In [8]:
import gc

def clear_mem():
    torch.cuda.empty_cache()
    gc.collect()

## Results

In [9]:
models = [
    "TheBloke/phi-2-GPTQ",
    "TheBloke/Llama-2-7B-GPTQ",
    "TheBloke/Llama-2-13B-GPTQ",
    "TheBloke/Mistral-7B-v0.1-GPTQ",
]


In [13]:
summaries = {}
data = []
for model_name in models:
    model, tokenizer = load_model(model_name)
    for sample in samples:
        if sample['name'] not in summaries:
            summaries[sample['name']] = summize(sample['text'])[:600]
        summary = summaries[sample['name']]

        # before 
        s1 = sample['text']
        results = perplexity_compute(data=s1, model=model, tokenizer=tokenizer, device='cuda')
        before = results['mean_perplexity']

        # after 
        s2 = f"""
        High level summary: {summary}

Text:
{sample['text']}
        """
        results = perplexity_compute(data=s2, model=model, tokenizer=tokenizer, device='cuda')
        after = np.array(results['perplexities'])[-len(s1):].mean()

        print(model_name, sample['name'], before, after)
        data.append(dict(before=before, after=after, model=model_name, sample=sample['name']))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using pad_token, but it is not set yet.
100%|██████████| 1/1 [00:00<00:00,  4.14it/s]


TheBloke/phi-2-GPTQ bad_ml 12.456705093383789 11.44649887084961


100%|██████████| 1/1 [00:00<00:00,  4.92it/s]


TheBloke/phi-2-GPTQ good_ml 22.6639461517334 20.115413665771484


100%|██████████| 1/1 [00:00<00:00,  3.81it/s]


...

TheBloke/Llama-2-7B-GPTQ email_to_fauci 11.317075729370117 10.221866607666016


100%|██████████| 1/1 [00:00<00:00,  3.44it/s]


TheBloke/Llama-2-7B-GPTQ enron_email1 13.783302307128906 11.154099464416504


100%|██████████| 1/1 [00:00<00:00,  2.73it/s]


TheBloke/Llama-2-7B-GPTQ openai_board_ann 5.916965007781982 5.880436897277832


In [ ]:
results

In [ ]:
# results
df = pd.DataFrame(data)
df["learning%"] = (df["before"] - df["after"])/df["before"]
# df

In [ ]:
for n,d in df.groupby("model"):
    print(n)
    d = d[['sample', 'learning%', 'in_training']].sort_values("learning%", ascending=False)
    print(d.to_markdown())
    display(d)

In [ ]:
# TODO: compare big and small